<a href="https://colab.research.google.com/github/ramjv/CNN-Model-optimize-using-Keras-Tuner/blob/master/fmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch

In [87]:
print(keras.__version__)
print(tf.__version__)

2.3.0-tf
2.2.0-rc2


In [88]:
batch_size = 128
num_classes = 10
epochs = 12

(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

img_rows, img_cols = 28, 28
train_images = train_images.reshape(len(train_images),img_rows,img_cols,1)
test_images = test_images.reshape(len(test_images),img_rows,img_cols,1)
input_shape = (img_rows, img_cols, 1)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

print('x_train shape:', train_images.shape)
print(train_images.shape[0], 'train samples')
print(test_images.shape[0], 'test samples')

train_labels = to_categorical(train_labels,num_classes)
test_labels = to_categorical(test_labels,num_classes)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [89]:

from kerastuner.engine.hyperparameters import HyperParameters
def build_model(hp):
  model = Sequential()
  model.add(Conv2D(filters=hp.Int('filter1',min_value=32,max_value=512,step=16),
                   kernel_size=hp.Choice('kernel_size1', values = [3,5]),
                   activation='relu',
                   input_shape=input_shape))
  model.add(Conv2D(filters=hp.Int('filter2',min_value=32,max_value=512,step=32),
                   kernel_size=hp.Choice('kernel_size2', values = [3,5]),
                   activation='relu'))
  #model.add(MaxPooling2D(pool_size=(2, 2)))
  #model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),activation='relu'))
  #model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),loss='categorical_crossentropy',metrics=['accuracy'])
  return model

tuner = RandomSearch(build_model,objective='val_accuracy',max_trials=5,executions_per_trial=1,directory='my_dir1',project_name='abcd')
tuner.search(train_images, train_labels,epochs=3,validation_split=0.1)
model=tuner.get_best_models(num_models=1)[0]
model.summary()


INFO:tensorflow:Reloading Oracle from existing project my_dir1/abcd/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir1/abcd/tuner0.json
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 256)       2560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 480)       3072480   
_________________________________________________________________
flatten (Flatten)            (None, 232320)            0         
_________________________________________________________________
dense (Dense)                (None, 96)                22302816  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 25,378,826
Trainable params: 25,378,826
Non-trainable params

In [90]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.1228 - accuracy: 0.9546 - val_loss: 0.2432 - val_accuracy: 0.9138
Epoch 5/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0800 - accuracy: 0.9708 - val_loss: 0.2987 - val_accuracy: 0.9172
Epoch 6/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0594 - accuracy: 0.9784 - val_loss: 0.3257 - val_accuracy: 0.9185
Epoch 7/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0410 - accuracy: 0.9854 - val_loss: 0.3688 - val_accuracy: 0.9175
Epoch 8/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0324 - accuracy: 0.9891 - val_loss: 0.4440 - val_accuracy: 0.9163
Epoch 9/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0299 - accuracy: 0.9892 - val_loss: 0.4933 - val_accuracy: 0.9135
Epoch 10/10
1688/1688 [==============================] - 35s 20ms/step - loss: 0.0227 - accuracy: 0.9921 - val_loss: 0.6167 